In [244]:
print("Hello,World")
import os
import pandas as pd
import glob
from functools import reduce




Hello,World


In [245]:

# Directory containing the Excel files
data_dir = r'C:\Users\yooji\Documents\Projects\24fall\explainable_fertility\data\사회학특강 발표 자료'
excel_files = glob.glob(os.path.join(data_dir, '*.xlsx'))

# Initialize list for reshaped DataFrames and dictionary for naming
reshaped_data_frames = []
dataframes = {}

# Load and process each Excel file
for file in excel_files:
    try:
        # Extracting the file name (without directory and extension) as key
        name = os.path.splitext(os.path.basename(file))[0]
        df = pd.read_excel(file)  # Load the Excel file into a DataFrame
        dataframes[name] = df  # Store the DataFrame in the dictionary
        reshaped_data_frames.append(df)  # Optionally keep it in a list
    except Exception as e:
        print(f"Error processing file {file}: {e}")

for name, df in dataframes.items():
    # 열 이름 변경
    if 'Unnamed: 0' in df.columns:
        df.rename(columns={'Unnamed: 0': '지역'}, inplace=True)
    # 필요 시 인덱스로 설정
    # df.set_index('지역', inplace=True)
dataframes['지역별 사교육비'] = dataframes['지역별 사교육비'].iloc[:, :-1]
# 제거된 결과 확인



In [246]:
# 공백과 특수문자 제거 함수
def clean_region_column(dataframes, region_column='지역'):
    for name, df in dataframes.items():
        if region_column in df.columns:
            # 문자열 공백 제거
            df[region_column] = df[region_column].str.strip()
            # 특수문자 (\xa0 등) 제거
            df[region_column] = df[region_column].str.replace(r'\xa0', '', regex=True)
            # print(f"{name}: 지역 열 정리 완료")
    return dataframes

# 모든 데이터프레임에 적용
dataframes = clean_region_column(dataframes)
# 명칭 매핑 딕셔너리
region_mapping = {
    '서울': '서울특별시',
    '광역시': ['부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시'],
    '대전': '대전광역시',
    '광주': '광주광역시',
    '부산': '부산광역시',
    '대구': '대구광역시',
    '인천': '인천광역시',
    '울산': '울산광역시',
    '세종': '세종특별자치시',
    '제주도': '제주특별자치도',
    '경기': '경기도',
    '강원': '강원특별자치도',
    '충북': '충청북도',
    '충남': '충청남도',
    '전라북도': '전북특별자치도',
    '전북': '전북특별자치도',
    '전남': '전라남도',
    '경북': '경상북도',
    '경남': '경상남도',
    '대도시(서울+광역시)': ['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시'],
    '대도시 이외(도 단위)': ['경기도', '강원특별자치도', '충청북도', '충청남도', '전북특별자치도', '전라남도', '경상북도', '경상남도', '제주특별자치도'],
}

# 모든 그룹화된 데이터를 개별 도시로 확장하기
expanded_mapping = {}
for key, value in region_mapping.items():
    if isinstance(value, list):
        for v in value:
            expanded_mapping[key] = value  # 그룹화된 값 추가
    else:
        expanded_mapping[key] = value  # 단일 값 처리


In [247]:
# 그룹화된 지역 확장을 위한 매핑
region_mapping = {
    '대도시(서울+ 광역시)': ['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시'],
    '중소 도시': ['경기도', '강원특별자치도', '충청북도', '충청남도', '전북특별자치도', '전라남도', '경상북도', '경상남도', '제주특별자치도'],
    '읍면지역': ['강원특별자치도', '충청북도', '충청남도', '전북특별자치도', '전라남도', '경상북도', '경상남도', '제주특별자치도']
}

# 확장 및 원래 행 삭제 함수
def expand_and_remove_grouped_regions(df, region_column='지역'):
    expanded_rows = []  # 확장된 행을 저장할 리스트
    for _, row in df.iterrows():
        region = row[region_column]
        if region in region_mapping:  # 그룹화된 지역인 경우
            for expanded_region in region_mapping[region]:
                new_row = row.copy()
                new_row[region_column] = expanded_region
                expanded_rows.append(new_row)
        else:  # 그룹화된 지역이 아닌 경우 그대로 유지
            expanded_rows.append(row)
    
    # 새로운 데이터프레임 생성 (그룹화된 원래 행 제거)
    expanded_df = pd.DataFrame(expanded_rows)
    expanded_df = expanded_df[~expanded_df[region_column].isin(region_mapping.keys())]
    return expanded_df

# '지역별 사교육비' 데이터프레임에 적용
dataframes['지역별 사교육비'] = expand_and_remove_grouped_regions(dataframes['지역별 사교육비'])
dataframes['시군구 합계 출산율 2015-2023']['지역'] = dataframes['시군구 합계 출산율 2015-2023']['지역'].replace('전라북도', '전북특별자치도')
remove_regions = ["서울", "광역시", "대도시 이외(도 단위)"]

# 특정 지역의 행 제거
dataframes['지역별 사교육비'] = dataframes['지역별 사교육비'][~dataframes['지역별 사교육비']['지역'].isin(remove_regions)]


In [248]:
for key, df in dataframes.items():
    if '지역' in df.columns:  # '지역' 열이 있는 경우에만 처리
        dataframes[key]['지역'] = df['지역'].replace('제주도', '제주특별자치도')


In [249]:
for d in dataframes:
    print(dataframes[d].head())

      지역   2015   2016   2017   2018   2019   2020   2021   2022
0  서울특별시  38911  41359  43731  46547  48248  49165  53373  56094
1  부산광역시  25707  26800  27210  28355  29436  29272  31950  34465
2  대구광역시  22685  23382  24170  24820  25653  25928  27770  29654
3  인천광역시  28376  29864  31198  31688  32272  32292  35388  38065
4  광주광역시  24731  26248  27449  28594  29807  30823  32098  33497
      지역  2015  2016  2017  2018  2019  2020  2021  2022
0  서울특별시  21.9  20.4  20.1  20.8  21.6  19.2  21.1  20.3
1  부산광역시  22.4  21.1  20.9  21.3  21.9  19.7  21.7  21.0
2  대구광역시  22.7  21.5  21.2  21.5  22.2  19.4  21.5  20.6
3  인천광역시  22.4  21.2  20.8  21.3  21.9  19.7  21.4  20.6
4  광주광역시  22.0  20.9  20.8  21.2  22.0  19.8  21.4  20.4
      지역   2015   2016   2017   2018   2019   2020   2021   2022
0  서울특별시  175.0  162.9  161.1  165.9  171.5  151.6  167.0  160.2
1  부산광역시  177.4  167.6  165.0  167.3  172.6  153.1  168.2  161.8
2  대구광역시  176.8  165.6  163.2  166.8  172.6  151.2  167.3  159.3
3  인천광역시

In [250]:
for name, df in dataframes.items():
    if '지역'  in df.columns:
        print(f"=== {name} ===")
        print("고유 지역 값:")
        print(df['지역'].unique())
        print("\n")
    elif '지역별'  in df.columns:
        print(f"=== {name} ===")
        print("고유 지역 값:")
        print(df['지역별'].unique())
        print("\n")
    else:
        print(f"=== {name} ===")
        print("해당 데이터프레임에 '지역' 열이 없습니다.\n")
    print(df.columns)


=== 1인당 GRDP 2015-2022 ===
고유 지역 값:
['서울특별시' '부산광역시' '대구광역시' '인천광역시' '광주광역시' '대전광역시' '울산광역시' '세종특별자치시' '경기도'
 '강원특별자치도' '충청북도' '충청남도' '전북특별자치도' '전라남도' '경상북도' '경상남도' '제주특별자치도']


Index(['지역', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022'], dtype='object')
=== 근로시간_상용근로일수 (일) ===
고유 지역 값:
['서울특별시' '부산광역시' '대구광역시' '인천광역시' '광주광역시' '대전광역시' '울산광역시' '경기도' '강원특별자치도'
 '충청북도' '충청남도' '전북특별자치도' '전라남도' '경상북도' '경상남도' '제주특별자치도']


Index(['지역', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022'], dtype='object')
=== 근로시간_상용소정실근로시간 (시간) ===
고유 지역 값:
['서울특별시' '부산광역시' '대구광역시' '인천광역시' '광주광역시' '대전광역시' '울산광역시' '경기도' '강원특별자치도'
 '충청북도' '충청남도' '전북특별자치도' '전라남도' '경상북도' '경상남도' '제주특별자치도']


Index(['지역', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022'], dtype='object')
=== 근로시간_상용월급여액 (원) ===
고유 지역 값:
['서울특별시' '부산광역시' '대구광역시' '인천광역시' '광주광역시' '대전광역시' '울산광역시' '경기도' '강원특별자치도'
 '충청북도' '충청남도' '전북특별자치도' '전라남도' '경상북도' '경상남도' '제주특별자치도']


Index(['지역', '2015', '2016', '2017', '201

In [251]:


# 열 이름을 모두 문자열로 변환하는 함수
def standardize_columns_to_str(dataframes):
    for name, df in dataframes.items():
        df.columns = [str(col) if isinstance(col, int) else col for col in df.columns]
        dataframes[name] = df  # 업데이트

# 적용
standardize_columns_to_str(dataframes)
# 수정된 melt_year_columns 함수
def melt_year_columns(df, value_name):
    # 열 이름을 모두 문자열로 변환
    df.columns = [str(col) if isinstance(col, int) else col for col in df.columns]
    
    # 연도 열 확인 (문자열로 변환된 연도)
    year_columns = [col for col in df.columns if col.isdigit()]
    
    # 데이터 변환
    melted_df = df.melt(id_vars=['지역'], value_vars=year_columns, var_name='연도', value_name=value_name)
    melted_df['연도'] = melted_df['연도'].astype(int)  # 연도를 숫자로 변환
    return melted_df

# 변환 적용
for name, df in dataframes.items():
    if any(col.isdigit() for col in df.columns):  # 열 이름에 문자열 숫자가 포함된 경우
        dataframes[name] = melt_year_columns(df, value_name=name)
        print(f"{name}: 변환 완료")


# 데이터 병합 함수 (상세 오류 메시지 추가)
def merge_dataframes(dataframes, merge_on=['지역', '연도']):
    merged_df = None

    for name, df in dataframes.items():
        # 공통 키 확인
        missing_keys = [key for key in merge_on if key not in df.columns]
        if missing_keys:
            print(f"'{name}' 병합 실패: 아래 키 누락 -> {missing_keys}")
            print(f"'{name}'의 열: {list(df.columns)}\n")
        else:
            print(f"병합 대상: {name}")
            if merged_df is None:
                merged_df = df  # 첫 번째 데이터프레임 설정
            else:
                # 병합
                merged_df = pd.merge(merged_df, df, on=merge_on, how='outer')

    return merged_df


# 병합 실행
merged_df = merge_dataframes(dataframes)



1인당 GRDP 2015-2022: 변환 완료
근로시간_상용근로일수 (일): 변환 완료
근로시간_상용소정실근로시간 (시간): 변환 완료
근로시간_상용월급여액 (원): 변환 완료
근로시간_상용정액급여 (원): 변환 완료
근로시간_상용초과근로시간 (시간): 변환 완료
근로시간_상용초과급여 (원): 변환 완료
근로시간_상용총근로시간 (시간): 변환 완료
근로시간_상용특별급여 (원): 변환 완료
시군구 합계 출산율 2015-2023: 변환 완료
지역별 1인가구 비율: 변환 완료
지역별 건강생활 실천율 2015-2023: 변환 완료
지역별 경제 참여율 2015-2023: 변환 완료
지역별 고용률 2015-2023: 변환 완료
지역별 고위험 음주율 2015-2023: 변환 완료
지역별 노령화 지수 215-2023: 변환 완료
지역별 문화기반시설 수 (인구 10만명 당) 2015-2022: 변환 완료
지역별 문화생활 인구 비율 2015-2023(2년 단위): 변환 완료
지역별 사교육비: 변환 완료
지역별 성평등 지수 2017-2022: 변환 완료
지역별 소득 2015-2022: 변환 완료
지역별 스트레스 인지율 2015-2023: 변환 완료
지역별 실업률 2015-2023: 변환 완료
지역별 여성 경제 참여율 2015-2023: 변환 완료
지역별 여성 고용률 2015-2023: 변환 완료
지역별 여성 실업률 2015-2023: 변환 완료
지역별 여성 청년층 (19-39) 비율 2015-2023: 변환 완료
지역별 여성 초혼 연령 2015-2023: 변환 완료
지역별 월평균 임금 2015-2023: 변환 완료
지역별 음주율 2015-2023: 변환 완료
지역별 인구밀도 2015-2023: 변환 완료
지역별 조이혼율 2015-2023: 변환 완료
지역별 조혼인율: 변환 완료
지역별 청년층(19-39) 비율 2015-2023: 변환 완료
지역별 평균 연령 2015-2023: 변환 완료
병합 대상: 1인당 GRDP 2015-2022
병합 대상: 근로시간_상용근로일수 (일)
병합 

In [252]:
# 'object' 타입 열 확인
object_cols = merged_df.select_dtypes(include=['object']).columns.tolist()
print("Object 타입 열:", object_cols)

# 각 'object' 열의 고유값 확인
for col in object_cols:
    print(f"\n{col}의 고유값:")
    print(merged_df[col].unique())

# 숫자형으로 변환 가능한 'object' 열 변환
# 예를 들어, '지역별 경제 참여율 2015-2023' 등은 숫자형일 가능성이 높습니다.
# 변환이 가능한 열은 float으로 변환하고, 불가능한 경우 범주형 변수로 처리
for col in object_cols:
    try:
        merged_df[col] = merged_df[col].astype(float)
        print(f"{col}을(float)으로 변환 성공.")
    except ValueError:
        print(f"{col}은(는) 변환할 수 없으므로 범주형으로 처리.")


Object 타입 열: ['지역', '지역별 경제 참여율 2015-2023', '지역별 고용률 2015-2023', '지역별 성평등 지수 2017-2022', '지역별 실업률 2015-2023', '지역별 여성 경제 참여율 2015-2023', '지역별 여성 고용률 2015-2023', '지역별 여성 실업률 2015-2023', '지역별 월평균 임금 2015-2023']

지역의 고유값:
['강원특별자치도' '경기도' '경상남도' '경상북도' '광주광역시' '대구광역시' '대전광역시' '부산광역시' '서울특별시'
 '세종특별자치시' '울산광역시' '인천광역시' '전라남도' '전북특별자치도' '제주특별자치도' '충청남도' '충청북도']

지역별 경제 참여율 2015-2023의 고유값:
[59.8 59.9 62.6 62.5 64.6 63.3 63.7 63.9 65.1 64.2 64.4 64.3 62.8 63.5
 65.7 62.2 62.7 63.1 63.2 63.6 63.0 64.5 65.2 60.3 60.8 61.7 61.5 61.0
 60.4 61.6 61.9 61.2 60.9 60.2 58.9 60.5 61.3 62.4 63.4 58.7 58.5 59.0
 58.1 58.8 58.2 59.6 62.1 62.3 '-' 64.0 65.0 65.9 61.8 60.7 61.1 64.9 64.7
 65.4 64.8 64.1 66.2 67.8 68.4 60.1 69.8 70.8 72.3 69.9 68.8 69.6 71.4
 70.6 65.6 65.8 66.5 65.3 67.0]

지역별 고용률 2015-2023의 고유값:
[57.9 58.2 61.0 60.7 62.3 60.8 61.4 61.8 63.3 61.7 62.1 62.0 61.9 60.3
 61.1 63.9 64.0 60.6 60.9 61.3 62.4 62.5 61.6 62.9 63.5 58.5 58.1 59.0
 59.4 59.3 58.7 58.6 60.1 58.8 56.6 59.1 59.8 59.6 59.7

In [253]:
merged_df.head()
# 데이터프레임의 기본 정보 확인
print(merged_df.info())

# 상위 몇 개 행 확인
print(merged_df.head())

# 통계적 요약 확인
print(merged_df.describe())
print("결측치 개수:\n", merged_df.isnull().sum())

# 결측치 비율 확인
print("결측치 비율:\n", merged_df.isnull().mean())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 37 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   지역                                  225 non-null    object 
 1   연도                                  225 non-null    int32  
 2   1인당 GRDP 2015-2022                  200 non-null    float64
 3   근로시간_상용근로일수 (일)                     192 non-null    float64
 4   근로시간_상용소정실근로시간 (시간)                 192 non-null    float64
 5   근로시간_상용월급여액 (원)                     192 non-null    float64
 6   근로시간_상용정액급여 (원)                     192 non-null    float64
 7   근로시간_상용초과근로시간 (시간)                  192 non-null    float64
 8   근로시간_상용초과급여 (원)                     192 non-null    float64
 9   근로시간_상용총근로시간 (시간)                   192 non-null    float64
 10  근로시간_상용특별급여 (원)                     192 non-null    float64
 11  시군구 합계 출산율 2015-2023                225 non-n

In [254]:
merged_df = merged_df[merged_df['지역'] != "세종특별자치시"]
# 지역별로 데이터를 그룹화하고 시간 기반 보간 수행
# '연도'를 기준으로 선형 보간 수행
def interpolate_by_region_linear(group):
    group = group.sort_values('연도')  # '연도' 기준으로 정렬
    group = group.interpolate(method='linear', limit_direction='forward')  # 선형 보간
    return group

# 지역별로 그룹화하고 보간 수행
interpolated_df = merged_df.groupby('지역').apply(interpolate_by_region_linear)

# 인덱스 정리
interpolated_df = interpolated_df.reset_index(drop=True)  # 기존 인덱스 리셋

# '지역별 성평등 지수 2017-2022' 컬럼의 결측치를 각 지역별 평균으로 채우기
interpolated_df['지역별 성평등 지수 2017-2022'] = (
    interpolated_df.groupby('지역')['지역별 성평등 지수 2017-2022']
    .transform(lambda group: group.fillna(group.mean()))
)



C:\Users\yooji\AppData\Local\Temp\ipykernel_4120\3269829124.py:6: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  group = group.interpolate(method='linear', limit_direction='forward')  # 선형 보간
C:\Users\yooji\AppData\Local\Temp\ipykernel_4120\3269829124.py:6: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  group = group.interpolate(method='linear', limit_direction='forward')  # 선형 보간
C:\Users\yooji\AppData\Local\Temp\ipykernel_4120\3269829124.py:6: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  group = group.interpolate(method='linear', limit_direction='forward')  # 선형 보간
C:\Users\yooji\AppData\Local\Temp\ipykernel_4120\

In [259]:
print(interpolated_df.isnull().sum())  # 모든 컬럼의 결측치 확인


지역                                    0
연도                                    0
1인당 GRDP 2015-2022                    0
근로시간_상용근로일수 (일)                       0
근로시간_상용소정실근로시간 (시간)                   0
근로시간_상용월급여액 (원)                       0
근로시간_상용정액급여 (원)                       0
근로시간_상용초과근로시간 (시간)                    0
근로시간_상용초과급여 (원)                       0
근로시간_상용총근로시간 (시간)                     0
근로시간_상용특별급여 (원)                       0
시군구 합계 출산율 2015-2023                  0
지역별 1인가구 비율                           0
지역별 건강생활 실천율 2015-2023                0
지역별 경제 참여율 2015-2023                  0
지역별 고용률 2015-2023                     0
지역별 고위험 음주율 2015-2023                 0
지역별 노령화 지수 215-2023                   0
지역별 문화기반시설 수 (인구 10만명 당) 2015-2022    0
지역별 문화생활 인구 비율 2015-2023(2년 단위)       0
지역별 사교육비                              0
지역별 성평등 지수 2017-2022                  0
지역별 소득 2015-2022                      0
지역별 스트레스 인지율 2015-2023                0
지역별 실업률 2015-2023                     0


In [260]:
merged_df.to_excel('final_data_outer.xlsx', index=False, engine='openpyxl')
interpolated_df.to_excel('data_interpolated.xlsx', index=False, engine='openpyxl')


In [257]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())
print("Current CUDA device:", torch.cuda.current_device())
print("CUDA device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")



CUDA available: False
Number of GPUs: 0


AssertionError: Torch not compiled with CUDA enabled